# Data Mashup - Create Time Series

This notebook will combine the following data sources into a country-year time series:

- UCDP/PRIO Armed Conflict
- Correlates of War
- Polity IV
- World Bank WDI

The year will range from 1946 - 2018, and the states will be uniquely identified by the variable 'country_name'.

In [1]:
import pandas as pd
import numpy as np

## Load base time series

In [2]:
base_ts = pd.read_csv("../Data/FINAL/countrycodes_ts-base.csv")
base_ts

,country_name,year,continent,region,cow_id,gw_id,iso3alpha,p4_id,wb_id
0,Afghanistan,1946,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG
1,Afghanistan,1947,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG
2,Afghanistan,1948,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG
3,Afghanistan,1949,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG
4,Afghanistan,1950,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG
...,...,...,...,...,...,...,...,...,...
10964,Zimbabwe,2014,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE
10965,Zimbabwe,2015,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE
10966,Zimbabwe,2016,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE
10967,Zimbabwe,2017,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE


## Incorporate UCDP/PRIO Armed Conflict data

All UCDP/PRIO variables get the column name prefix 'ucdp_'

In [3]:
ucdp = pd.read_csv("../Data/FINAL/ucdp.csv")
ucdp = ucdp.add_prefix('ucdp_')
ucdp = ucdp.rename(columns={'ucdp_gw_id': 'gw_id', 'ucdp_year': 'year'})
ucdp

,gw_id,year,ucdp_war_count,ucdp_conflict_count,ucdp_max_ep-intensity,ucdp_incompatibility_isTerritory,ucdp_incompatibility_isGovernment,ucdp_type_Extrasystemic,ucdp_type_Internal,ucdp_type_InternationalizedInternal,ucdp_type_Interstate,ucdp_role_primary,ucdp_role_secondary,ucdp_ep-intensity_Minor,ucdp_ep-intensity_War,ucdp_cum-intensity_Minor,ucdp_cum-intensity_War
0,2,1950,1,2,2,2,0,1,0,0,1,1,1,1,1,1,1
1,2,1951,1,1,2,1,0,0,0,0,1,0,1,0,1,0,1
2,2,1952,1,1,2,1,0,0,0,0,1,0,1,0,1,0,1
3,2,1953,1,1,2,1,0,0,0,0,1,0,1,0,1,0,1
4,2,1954,1,1,2,1,1,0,0,0,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2844,972,2008,1,1,2,0,1,0,1,0,0,0,1,0,1,0,1
2845,972,2011,1,1,2,0,1,0,1,0,0,0,1,0,1,0,1
2846,972,2012,1,1,2,0,1,0,1,0,0,0,1,0,1,0,1
2847,972,2013,1,1,2,0,1,0,1,0,0,0,1,0,1,0,1


For the war count and conflict count columns, need to fill in a 0 for years/states where there was no conflict. For the remaining ucdp variables, if there was no conflict that year they will remain NaNs.

In [4]:
ts_merge1 = base_ts.merge(ucdp, on=['gw_id', 'year'], how='left')
ts_merge1['ucdp_war_count'] = ts_merge1['ucdp_war_count'].fillna(0).astype(int)
ts_merge1['ucdp_conflict_count'] = ts_merge1['ucdp_conflict_count'].fillna(0).astype(int)
ts_merge1

,country_name,year,continent,region,cow_id,gw_id,iso3alpha,p4_id,wb_id,ucdp_war_count,...,ucdp_type_Extrasystemic,ucdp_type_Internal,ucdp_type_InternationalizedInternal,ucdp_type_Interstate,ucdp_role_primary,ucdp_role_secondary,ucdp_ep-intensity_Minor,ucdp_ep-intensity_War,ucdp_cum-intensity_Minor,ucdp_cum-intensity_War
0,Afghanistan,1946,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1947,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1948,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1949,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1950,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10964,Zimbabwe,2014,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10965,Zimbabwe,2015,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10966,Zimbabwe,2016,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10967,Zimbabwe,2017,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Incorporate CoW data

All CoW variables get the prefix 'cow_'

In [5]:
cow = pd.read_csv("../Data/FINAL/cow.csv")
cow = cow.add_prefix('cow_')
cow = cow.rename(columns={'cow_cow_id': 'cow_id', 'cow_year': 'year'})
cow

,cow_id,year,cow_war_count,cow_IsInitiator,cow_avg_deaths,cow_type_extrastate,cow_type_interstate,cow_type_intrastate,cow_type_nonstate,cow_outcome_cont-conflict,cow_outcome_lose,cow_outcome_ongoing2008,cow_outcome_stalemate,cow_outcome_tied,cow_outcome_war-transitioned,cow_outcome_win
0,2,1950,1,0,13622,0,1,0,0,0,0,0,1,0,0,0
1,2,1951,1,0,13622,0,1,0,0,0,0,0,1,0,0,0
2,2,1952,1,0,13622,0,1,0,0,0,0,0,1,0,0,0
3,2,1953,1,0,13622,0,1,0,0,0,0,0,1,0,0,0
4,2,1958,1,0,1,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,900,2008,2,2,1,2,0,0,0,0,0,2,0,0,0,0
1238,910,1989,1,1,0,0,0,1,0,1,0,0,0,0,0,0
1239,910,1990,1,1,0,0,0,1,0,1,0,0,0,0,0,0
1240,910,1991,1,1,0,0,0,1,0,1,0,0,0,0,0,0


Need to fill in 0s for country-years with no conflict for the war count variable. Years after 2007 should remain NA as there is no data from CoW for these years. Other CoW columns remain NA for years when there is no conflict.

In [6]:
ts_merge2 = ts_merge1.merge(cow, on=['cow_id', 'year'], how='left')
ts_merge2['cow_war_count'] [ts_merge2['year'] < 2008] = ts_merge2['cow_war_count'].fillna(0).astype('Int64')
ts_merge2

<ipython-input-6-23a887262a75>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts_merge2['cow_war_count'] [ts_merge2['year'] < 2008] = ts_merge2['cow_war_count'].fillna(0).astype('Int64')


,country_name,year,continent,region,cow_id,gw_id,iso3alpha,p4_id,wb_id,ucdp_war_count,...,cow_type_interstate,cow_type_intrastate,cow_type_nonstate,cow_outcome_cont-conflict,cow_outcome_lose,cow_outcome_ongoing2008,cow_outcome_stalemate,cow_outcome_tied,cow_outcome_war-transitioned,cow_outcome_win
0,Afghanistan,1946,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1947,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1948,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1949,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1950,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10964,Zimbabwe,2014,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10965,Zimbabwe,2015,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10966,Zimbabwe,2016,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10967,Zimbabwe,2017,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Incorporate Polity data

All columns from polity get the prefix 'p4_'

In [7]:
polity = pd.read_csv("../Data/FINAL/polity.csv")
polity = polity.add_prefix('p4_')
polity = polity.rename(columns={'p4_ccode': 'p4_id', 'p4_year': 'year'})
polity

,p4_id,year,p4_fragment,p4_democ,p4_autoc,p4_polity,p4_polity2,p4_durable,p4_xrreg,p4_xrcomp,...,p4_parreg,p4_parcomp,p4_exrec,p4_exconst,p4_polcomp,p4_interim,p4_change,p4_d4,p4_sf,p4_regtrans
0,2,1946,NaN,10,0,10,10.0,137.0,3,3,...,5,5,8.0,7,10.0,NaN,NaN,NaN,NaN,NaN
1,2,1947,NaN,10,0,10,10.0,138.0,3,3,...,5,5,8.0,7,10.0,NaN,NaN,NaN,NaN,NaN
2,2,1948,NaN,10,0,10,10.0,139.0,3,3,...,5,5,8.0,7,10.0,NaN,NaN,NaN,NaN,NaN
3,2,1949,NaN,10,0,10,10.0,140.0,3,3,...,5,5,8.0,7,10.0,NaN,NaN,NaN,NaN,NaN
4,2,1950,NaN,10,0,10,10.0,141.0,3,3,...,5,5,8.0,7,10.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9959,950,2014,0.0,3,1,2,2.0,0.0,2,0,...,3,4,5.0,4,8.0,NaN,6.0,1.0,NaN,3.0
9960,950,2015,0.0,3,1,2,2.0,1.0,2,0,...,3,4,5.0,4,8.0,NaN,NaN,NaN,NaN,NaN
9961,950,2016,0.0,3,1,2,2.0,2.0,2,0,...,3,4,5.0,4,8.0,NaN,NaN,NaN,NaN,NaN
9962,950,2017,0.0,3,1,2,2.0,3.0,2,0,...,3,4,5.0,4,8.0,NaN,NaN,NaN,NaN,NaN


In [8]:
ts_merge3 = ts_merge2.merge(polity, on=['p4_id', 'year'], how='left')
ts_merge3

,country_name,year,continent,region,cow_id,gw_id,iso3alpha,p4_id,wb_id,ucdp_war_count,...,p4_parreg,p4_parcomp,p4_exrec,p4_exconst,p4_polcomp,p4_interim,p4_change,p4_d4,p4_sf,p4_regtrans
0,Afghanistan,1946,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,4.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1947,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,4.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1948,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,4.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1949,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,4.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1950,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,4.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10964,Zimbabwe,2014,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,3.0,3.0,7.0,5.0,6.0,NaN,NaN,NaN,NaN,NaN
10965,Zimbabwe,2015,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,3.0,3.0,7.0,5.0,6.0,NaN,NaN,NaN,NaN,NaN
10966,Zimbabwe,2016,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,3.0,3.0,7.0,5.0,6.0,NaN,NaN,NaN,NaN,NaN
10967,Zimbabwe,2017,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,3.0,3.0,7.0,5.0,6.0,NaN,NaN,NaN,NaN,NaN


## Incorporate World Bank data

All columns from the World Bank get the prefix 'wb_'. All variable names are the indicator IDs. NaN values remain NaN, so individual researchers can deal with missing values as they prefer.

In [9]:
wdi = pd.read_csv("../Data/FINAL/wdi_top25.csv")
wdi

,country,indicator,year,value,decimal
0,AFG,NY.GDP.MKTP.CD,2019,NaN,0
1,AFG,NY.GDP.MKTP.CD,2018,1.936297e+10,0
2,AFG,NY.GDP.MKTP.CD,2017,2.019176e+10,0
3,AFG,NY.GDP.MKTP.CD,2016,1.936264e+10,0
4,AFG,NY.GDP.MKTP.CD,2015,1.990711e+10,0
...,...,...,...,...,...
290995,ZWE,SP.DYN.IMRT.IN,1964,8.320000e+01,0
290996,ZWE,SP.DYN.IMRT.IN,1963,8.570000e+01,0
290997,ZWE,SP.DYN.IMRT.IN,1962,8.810000e+01,0
290998,ZWE,SP.DYN.IMRT.IN,1961,9.050000e+01,0


Need to pivot the original data so that each indicator becomes a column. As a result the "decimal" value (which may be 0, 1, or 2) is lost. If the decimal value needs to be incorporated (e.g. value * 10^decimal), this should be done before the pivot.

In [10]:
wdi_pivot = wdi.pivot_table(index=['country', 'year'], columns='indicator', values='value')
wdi_pivot = wdi_pivot.add_prefix('wb_')
wdi_pivot = wdi_pivot.reset_index()
wdi_pivot = wdi_pivot.rename(columns={'country': 'wb_id'})
wdi_pivot

indicator,wb_id,year,wb_BX.KLT.DINV.CD.WD,wb_EG.ELC.ACCS.ZS,wb_EN.ATM.CO2E.PC,wb_EN.POP.DNST,wb_FP.CPI.TOTL.ZG,wb_MS.MIL.XPND.GD.ZS,wb_NE.EXP.GNFS.ZS,wb_NY.GDP.MKTP.CD,...,wb_SI.POV.GINI,wb_SL.UEM.TOTL.ZS,wb_SP.DYN.IMRT.IN,wb_SP.DYN.LE00.IN,wb_SP.DYN.TFRT.IN,wb_SP.POP.GROW,wb_SP.POP.TOTL,wb_SP.POP.TOTL.FE.IN,wb_SP.URB.TOTL.IN.ZS,wb_ST.INT.ARVL
0,AFG,1960,NaN,NaN,0.046057,NaN,NaN,NaN,4.132233,5.377778e+08,...,NaN,NaN,NaN,32.446,7.450,1.828639,8996973.0,4347397.0,8.401,NaN
1,AFG,1961,NaN,NaN,0.053589,14.044987,NaN,NaN,4.453443,5.488889e+08,...,NaN,NaN,236.5,32.962,7.450,1.898476,9169410.0,4439158.0,8.684,NaN
2,AFG,1962,NaN,NaN,0.073721,14.323808,NaN,NaN,4.878051,5.466667e+08,...,NaN,NaN,232.6,33.471,7.450,1.965751,9351441.0,4535392.0,8.976,NaN
3,AFG,1963,NaN,NaN,0.074161,14.617537,NaN,NaN,9.171601,7.511112e+08,...,NaN,NaN,228.9,33.971,7.450,2.029893,9543205.0,4636172.0,9.276,NaN
4,AFG,1964,NaN,NaN,0.086174,14.926295,NaN,NaN,8.888893,8.000000e+08,...,NaN,NaN,225.1,34.463,7.450,2.090248,9744781.0,4741531.0,9.586,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11598,ZWE,2015,3.992000e+08,33.700000,NaN,35.710557,-2.409500,2.343629,19.160176,1.996312e+10,...,NaN,5.438,38.5,59.534,3.896,1.663813,13814629.0,7245857.0,32.385,2057000.0
11599,ZWE,2016,3.430138e+08,39.892345,NaN,36.268295,-1.566413,2.220728,19.943532,2.054868e+10,...,NaN,5.239,36.3,60.294,3.804,1.549759,14030390.0,7356159.0,32.296,2168000.0
11600,ZWE,2017,2.471877e+08,40.421368,NaN,36.801719,0.909733,1.930134,18.993548,2.281301e+10,...,NaN,4.943,35.4,60.812,3.707,1.460061,14236745.0,7459621.0,32.237,2423000.0
11601,ZWE,2018,7.446372e+08,NaN,NaN,37.324591,NaN,2.169606,22.924114,3.100052e+10,...,NaN,4.915,33.9,NaN,NaN,1.410783,14439018.0,7559801.0,32.209,NaN


In [11]:
ts_merge4 = ts_merge3.merge(wdi_pivot, how='left', on=['wb_id', 'year'])
ts_merge4

,country_name,year,continent,region,cow_id,gw_id,iso3alpha,p4_id,wb_id,ucdp_war_count,...,wb_SI.POV.GINI,wb_SL.UEM.TOTL.ZS,wb_SP.DYN.IMRT.IN,wb_SP.DYN.LE00.IN,wb_SP.DYN.TFRT.IN,wb_SP.POP.GROW,wb_SP.POP.TOTL,wb_SP.POP.TOTL.FE.IN,wb_SP.URB.TOTL.IN.ZS,wb_ST.INT.ARVL
0,Afghanistan,1946,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1947,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1948,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1949,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1950,Asia,Southern Asia,700.0,700.0,AFG,700.0,AFG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10964,Zimbabwe,2014,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,NaN,5.518,40.4,58.410,3.974,1.754692,13586681.0,7126780.0,32.504,1880000.0
10965,Zimbabwe,2015,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,NaN,5.438,38.5,59.534,3.896,1.663813,13814629.0,7245857.0,32.385,2057000.0
10966,Zimbabwe,2016,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,NaN,5.239,36.3,60.294,3.804,1.549759,14030390.0,7356159.0,32.296,2168000.0
10967,Zimbabwe,2017,Africa,Eastern Africa,552.0,552.0,ZWE,552.0,ZWE,0,...,NaN,4.943,35.4,60.812,3.707,1.460061,14236745.0,7459621.0,32.237,2423000.0


## Export final time-series

Each row is uniquely identified by the 'country_name' and 'year' variables.

In [12]:
ts_merge4.duplicated(subset=['country_name', 'year']).sum()

0

In [13]:
ts_merge4.to_csv("../Data/FINAL/final_time-series.csv", index=False, encoding='utf-8')